In [6]:
# e.g.: python drunk_detector data --train-files data/train/*/* --test-files data/test/*/* --val-files data/validation/*/*
#       python drunk_detector train -d data_2020-03-18_20-59-45.pickle

import argparse
import datetime
import numpy as np
import os
import pickle
from PIL import Image
import re
import itertools
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

from imgaug import augmenters as iaa

# Model types
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Tensorflow Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint


class DrunkDetector:
    def __init__(self, args):
        self.face_filename_pattern = '[0-4][0-9]_[a-z]*_[0-4]_f_[FM]_[0-9_]*\.tif'
        self.sober_filename_patter = '[0-4][0-9]_[a-z]*_1_f_[FM]_[0-9_]*\.tif'
        self.args = args
        self.data = { 'train': [], 'test': [], 'val': [] }

    def format_data(self):
        self.read_images(self.args.train_files, 'train')
        self.read_images(self.args.test_files, 'test')
        self.read_images(self.args.val_files, 'val')
        self.augment_data('train')
        self.augment_data('val')
        self.augment_data('test')
        curr_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        filename = '{}_{}.pickle'.format('data', curr_datetime)
        with open(os.path.join(self.args.output_dir, filename), 'wb') as out_file:
            pickle.dump(self.data, out_file)

    def augment_data(self, split_set):
        augmented_data = []
        for datum in self.data[split_set]:
            # horizontal flip with a noise on non-zero values
            variance = [[[20 if col else 0 for col in row] for row in layer ] for layer in datum['thermal_frames']]
            thermal_frames = np.random.normal(np.flip(datum['thermal_frames'], 2), variance)
            thermal_sum = np.zeros((128, 160))
            for f, frame in enumerate(thermal_frames):
                min_val = max(np.amin(frame), 0)
                frame -= min_val
                for r, row in enumerate(frame):
                    for c, val in enumerate(row):
                        if val < 0:
                            val = 0
                            thermal_frames[f, r, c] = 0
                        thermal_sum[r, c] += val
            augmented_data.append({
                'filename': datum['filename'] + '_flipped',
                'thermal_sum': thermal_sum,
                'thermal_frames': thermal_frames,
                'y': datum['y']
            })

            # noisy version of image
            variance = [[[100 if col else 0 for col in row] for row in layer ] for layer in datum['thermal_frames']]
            thermal_frames = np.random.normal(datum['thermal_frames'], variance)
            thermal_sum = np.zeros((128, 160))
            for f, frame in enumerate(thermal_frames):
                min_val = max(np.amin(frame), 0)
                frame -= min_val
                for r, row in enumerate(frame):
                    for c, val in enumerate(row):
                        if val < 0:
                            val = 0
                            thermal_frames[f, r, c] = 0
                        thermal_sum[r, c] += val
            augmented_data.append({
                'filename': datum['filename'] + '_noisy',
                'thermal_sum': thermal_sum,
                'thermal_frames': thermal_frames,
                'y': datum['y']
            })

            # blurred version of image
            thermal_frames = gaussian_filter(datum['thermal_frames'], sigma=3)
            thermal_sum = np.zeros((128, 160))
            for _, frame in enumerate(thermal_frames):
                min_val = max(np.amin(frame), 0)
                frame -= min_val
                for r, row in enumerate(frame):
                    for c, val in enumerate(row):
                        thermal_sum[r, c] += val
            augmented_data.append({
                'filename': datum['filename'] + '_blur',
                'thermal_sum': thermal_sum,
                'thermal_frames': thermal_frames,
                'y': datum['y']
            })

            # im = Image.fromarray(thermal_sum/np.max(thermal_sum) * 255)
            # im.show()
            # im_orig = Image.fromarray(datum['thermal_sum']/np.max(datum['thermal_sum']) * 255)
            # im_orig.show()
            # im_orig.show()
            # breakpoint()
        self.data[split_set] += augmented_data

    def read_images(self, files, split_set):
        for filename in files:
            basename = os.path.basename(filename)
            if not self.is_face_image(basename):
                continue

            with Image.open(filename) as img:
                self.data[split_set].append({
                    'filename': basename,
                    'thermal_sum': np.zeros((128, 160)),
                    'thermal_frames': np.zeros((img.n_frames, 128, 160)),
                    'y': self.is_sober_image(basename)
                })

                for i in range(img.n_frames):
                    img.seek(i)
                    frame_data = np.array(img)
                    min_val = np.amin(frame_data)
                    frame_data -= min_val
                    for j in range(img.height):
                        for k in range(img.width):
                            self.data[split_set][-1]['thermal_sum'][j, k] += frame_data[j, k]

                    self.data[split_set][-1]['thermal_frames'][i] = frame_data

    def is_face_image(self, filename):
        return re.match(self.face_filename_pattern, filename) is not None

    def is_sober_image(self, filename):
        return re.match(self.sober_filename_patter, filename) is not None

    def train(self):
        if self.args.data is None:
            print('Error: No data given.')
            exit(1)

        with open(self.args.data, 'rb') as data_file:
            self.data = pickle.load(data_file)

        if False:
            self.train_X = np.array([datum['thermal_frames'] for datum in self.data['train']])
            self.val_X = np.array([datum['thermal_frames'] for datum in self.data['val']])
            self.test_X = np.array([datum['thermal_frames'] for datum in self.data['test']])
        elif True:
            self.train_X = np.array([datum['thermal_sum'] for datum in self.data['train']])
            self.val_X = np.array([datum['thermal_sum'] for datum in self.data['val']])
            self.test_X = np.array([datum['thermal_sum'] for datum in self.data['test']])

        self.train_X_2d = np.array([datum.flatten() for datum in self.train_X])
        self.val_X_2d = np.array([datum.flatten() for datum in self.val_X])
        self.test_X_2d = np.array([datum.flatten() for datum in self.test_X])

        self.train_y = np.array([datum['y'] for datum in self.data['train']])
        self.val_y = np.array([datum['y'] for datum in self.data['val']])
        self.test_y = np.array([datum['y'] for datum in self.data['test']])
        
        # Horizontal flip
        flip_seq = iaa.Sequential([
            iaa.Fliplr(1), # horizontally flip all of the images
        ])
        flip_data = flip_seq(images=self.train_X)
        self.train_X = np.concatenate((self.train_X, flip_data), axis=0)
        self.train_y = np.concatenate((self.train_y, self.train_y), axis=0)
        flip_data = flip_seq(images=self.val_X)
        self.val_X = np.concatenate((self.val_X, flip_data), axis=0)
        self.val_y = np.concatenate((self.val_y, self.val_y), axis=0)
        flip_data = flip_seq(images=self.test_X)
        self.test_X = np.concatenate((self.test_X, flip_data), axis=0)
        self.test_y = np.concatenate((self.test_y, self.test_y), axis=0)
        
        # Shuffle images
        self.train_X, self.train_y = shuffle_pair(self.train_X, self.train_y)
        self.val_X, self.val_y = shuffle_pair(self.val_X, self.val_y)
        self.test_X, self.test_y = shuffle_pair(self.test_X, self.test_y)

        # self.train_svm()
        # self.train_rf()
        # self.train_lr()
        # self.train_sgd()
        # self.train_knn()
        # self.train_dt()
        # self.train_mlp()
        self.train_cnn_h()

    # Decision Tree
    def train_dt(self):
        dt = DecisionTreeClassifier()
        dt.fit(self.train_X_2d, self.train_y)
        pred_y = dt.predict(self.val_X_2d)
        print('Decision Tree accuracy:', accuracy_score(self.val_y, pred_y))

    # K Nearest Neighbors
    def train_knn(self):
        knn = KNeighborsClassifier()
        knn.fit(self.train_X_2d, self.train_y)
        pred_y = knn.predict(self.val_X_2d)
        print('KNN accuracy:', accuracy_score(self.val_y, pred_y))

    # Logistic Regression
    def train_lr(self):
        lr = LogisticRegression(max_iter=200)
        lr.fit(self.train_X_2d, self.train_y)
        pred_y = lr.predict(self.val_X_2d)
        print('logistic regression accuracy:', accuracy_score(self.val_y, pred_y))

    # Multi-Layer Perceptron
    def train_mlp(self):
        parameters = {
            'learning_rate': ['constant', 'invscaling', 'adaptive'],
            'activation': ['identity', 'logistic', 'tanh', 'relu'],
            'solver': ['lbfgs', 'sgd', 'adam']
            # 'hidden_layer_sizes': []
        }
        mlp = MLPClassifier()
        clf = GridSearchCV(mlp, parameters)
        clf.fit(self.train_X_2d, self.train_y)
        pred_y = clf.predict(self.val_X_2d)
        print('Multi-Layer Perceptron accuracy:', accuracy_score(self.val_y, pred_y))

    # Random Forest
    def train_rf(self):
        parameters = {
            'n_estimators': [50, 100, 250, 500], # 250
            'min_impurity_decrease': [0, 0.25, 0.5], # 0
            'class_weight': [None, 'balanced'] # balanced
        }
        rfc = RandomForestClassifier()
        clf = GridSearchCV(rfc, parameters)
        clf.fit(self.train_X_2d, self.train_y)
        pred_y = clf.predict(self.val_X_2d)
        print('random forest accuracy:', accuracy_score(self.val_y, pred_y))
        print('\tparams:', clf.best_params_)

    # Stochastic Gradient Descent
    def train_sgd(self):
        sgd = SGDClassifier()
        sgd.fit(self.train_X_2d, self.train_y)
        pred_y = sgd.predict(self.val_X_2d)
        print('Stochastic Gradient Descent accuracy:', accuracy_score(self.val_y, pred_y))
 
    # Support Vector Machine
    def train_svm(self):
        svc = SVC()
        svc.fit(self.train_X_2d, self.train_y)
        pred_y = svc.predict(self.val_X_2d)
        print('svm accuracy:', accuracy_score(self.val_y, pred_y))

    # Convolutional Neural Network
    def train_cnn(self):
        #assert self.args.data_mode == 'sum', \
                #'Use [-m sum] for training CNN'
            
        self.train_X = tf.keras.utils.normalize(self.train_X, axis=1)
        self.val_X = tf.keras.utils.normalize(self.val_X, axis=1)
        self.test_X = tf.keras.utils.normalize(self.test_X, axis=1)
        
        curr_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

        x, y = self.train_X.shape[1:]

        model = Sequential()
        # Layer 1
        model.add(Conv2D(8, (3, 3), input_shape=(x,y,1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        # Layer 2
        model.add(Conv2D(16, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        # Layer 3
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        # Output layer
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization())
        
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        model.compile(loss='binary_crossentropy',
                        optimizer=optimizers.Adam(learning_rate=args.learning_rate),
                        metrics=['accuracy'])

        # Treat every sober image with same weight as 3 drunk images
        # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#class_weights
        class_weight = {0: (1/4)/2, 1: (3/4)/2}

        #### Below is adapted from class example of CNN
        num_epochs = 1000

        # Holds performance statistics across epochs
        perf_time = np.zeros((num_epochs, 4))

        # Set up figure
        fig = plt.figure()
        ax1 = fig.add_subplot(111)

        best_val = [np.inf, 0]
        for epoch in np.arange(0,num_epochs):
            model.fit(cnn_data(self.train_X), np.array(self.train_y),
                      batch_size=32,
                      epochs=1,
                      verbose=1,
                      class_weight=class_weight,
                      validation_data=(cnn_data(self.val_X), np.array(self.val_y)),
                      shuffle=True,
                      use_multiprocessing=True)
            # Check the performance on train/test/val
            # The model.evaluate function returns an array: [loss, accuracy]
            val = model.evaluate(cnn_data(self.val_X), np.array(self.val_y))  # val = [val_loss, val_accuracy]
            new = [model.evaluate(cnn_data(self.train_X), np.array(self.train_y))[1],
                   val[0], val[1],
                   model.evaluate(cnn_data(self.test_X), np.array(self.test_y))[1]]
            perf_time[epoch,:]=new

            # Visualize
            plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,0],'b', label='train')
            plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,2],'r', label='validation')
            plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,3],'g', label='test')
            plt.legend(loc='upper left')
            plt.show()

            # Test if validation performance has improved (val_loss)
            if val[0] >= best_val[0]:
                best_val[1] += 1
            # Model improved
            else:
                best_val = [val[0], 0]
                # Save current best model
                filename_model = '{}_{}_epoch={}_val_acc={}.hdf5'.format('cnn_model', curr_datetime, epoch, val[1])
                save_model(model, filename_model)
            print ("epoch %d, loss %f, number %d" %(epoch, best_val[0], best_val[1]))

            # Stop training if performance hasn't increased in STOP_ITERATIONS
            STOP_ITERATIONS = 30
            if best_val[1] > STOP_ITERATIONS:
                break

        # Export model and test/val/train plot
        fig = plt.figure()
        ax1 = fig.add_subplot(111)
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,0],'b', label='train')
        plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,2],'r', label='validation')
        plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,3],'g', label='test')
        plt.legend(loc='upper left')

        filename_fig = '{}_{}.png'.format('cnn_fig', curr_datetime)
        plt.savefig(filename_fig)
        plt.close('all') # Close fig to save memory

        filename_model = '{}_{}.hdf5'.format('cnn_model', curr_datetime)
        save_model(model, filename_model)

        pred_y = model.predict_classes(cnn_data(self.test_X))
        print('CNN accuracy:', accuracy_score(self.test_y, pred_y))
        print('CNN confusion matrix\n', confusion_matrix(self.test_y, pred_y))
        
        self.model = model
        
        
        
        
        
        
        
        
        
        
        
        
        
        
    # Convolutional Neural Network hyperparameter tuning
    def train_cnn_h(self):
        #assert self.args.data_mode == 'sum', \
                #'Use [-m sum] for training CNN'
            
        self.train_X = tf.keras.utils.normalize(self.train_X, axis=1)
        self.val_X = tf.keras.utils.normalize(self.val_X, axis=1)
        self.test_X = tf.keras.utils.normalize(self.test_X, axis=1)
        
        curr_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

        x, y = self.train_X.shape[1:]

        
        # saved_acc = pickle.load(open('saved_acc.pickle, 'rb'))
        
        # hyperparameters
        num_convlayers = [1, 2, 3]
        convlayer_size = [8, 16, 32]
        num_denselayers = [0, 1, 2]
        denselayer_size = [128, 256, 512]
        learning_rate = [0.01, 0.001, 0.0001]
        batch_size = [16, 32]
        # batch_normalization = [True, False]
        
        # creates a list of all combinations of hyperparameters
        param_grid = list(itertools.product(
            num_convlayers, convlayer_size, 
            num_denselayers, denselayer_size,
            learning_rate, 
            batch_size))

        print(f'CNN hyperparameter tuning with {len(param_grid)} combinations')
        
        # offset = len(saved_acc)
        for i, params in enumerate(shuffle(param_grid)):
            
            print(f'Run {i}/{len(param_grid)}')
            
            # tuning parameters
            num_convlayers, \
            convlayer_size, \
            num_denselayers, \
            denselayer_size, \
            learning_rate, \
            batch_size = params
            
            NAME = f'n_clayers={num_convlayers},clayer_sz={convlayer_size},n_dlayers={num_denselayers},dlayer_sz={denselayer_size},lr={learning_rate},bat_size={batch_size}'
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
#             save_best_model_loss = ModelCheckpoint("val_loss={val_loss:.4f}"+f"{NAME}.hdf5", 
#                                                   monitor='val_loss', 
#                                                   verbose=0, save_best_only=True, 
#                                                   save_weights_only=False, 
#                                                   mode='auto', save_freq='epoch')
#             save_best_model_acc = tf.keras.callbacks.ModelCheckpoint("val_acc={val_accuracy:.4f}"+f"{NAME}.hdf5", 
#                                                      monitor='val_accuracy', 
#                                                      verbose=0, save_best_only=True, 
#                                                      save_weights_only=False, 
#                                                      mode='auto', save_freq='epoch')
            early_stopping = tf.keras.callbacks.EarlyStopping(
                                                    monitor='val_accuracy', 
                                                    verbose=1,
                                                    patience=30,
                                                    mode='auto',
                                                    restore_best_weights=False)
            
            model = Sequential()
            # Conv layer 1
            model.add(Conv2D(convlayer_size, (3, 3), input_shape=(x,y,1)))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            
            # Additional Conv layers
            for _ in range(num_convlayers-1):
                model.add(Conv2D(convlayer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            # Dense layer 1
            model.add(Flatten())
            for _ in range(num_denselayers):
                model.add(Dense(denselayer_size, activation='relu'))
            
            if False: 
                model.add(BatchNormalization())

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                            optimizer=optimizers.Adam(learning_rate=learning_rate),
                            metrics=['accuracy'])

            # Treat every sober image with same weight as 3 drunk images
            # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#class_weights
            class_weight = {0: (1/4)/2, 1: (3/4)/2}

            num_epochs = 1000
            model.fit(cnn_data(self.train_X), np.array(self.train_y),
                      batch_size=batch_size,
                      epochs=num_epochs,
                      verbose=1,
                      class_weight=class_weight,
                      validation_data=(cnn_data(self.val_X), np.array(self.val_y)),
                      shuffle=True,
                      use_multiprocessing=True,
                      callbacks=[tensorboard, early_stopping])

#             filename_model = f'n_clayers={num_convlayers},clayer_sz={convlayer_size},n_dlayers={num_denselayers},dlayer_sz={denselayer_size},lr={learning_rate},bat_size={batch_size},bat_norm={batch_normalization}.hdf5'
#             save_model(model, filename_model)

#             pred_y = model.predict_classes(cnn_data(self.test_X))
#             print('CNN accuracy:', accuracy_score(self.test_y, pred_y))
#             print('CNN confusion matrix\n', confusion_matrix(self.test_y, pred_y))
        
        # self.model = model
        
# Shuffles a pair of equal size arrays in the same random order
# Returns a tuple of shuffled arrays
def shuffle_pair(X_arr, y_arr):
    X_arr_s = []
    y_arr_s = []
    for X, y in shuffle(list(zip(X_arr, y_arr))):
        X_arr_s.append(X)
        y_arr_s.append(y)
    return (np.array(X_arr_s), np.array(y_arr_s))

# Only for X data, use np.array(y) for y data
# Returns data formatted for Keras CNN input
def cnn_data(data):
    x, y = data.shape[1:]
    return data.reshape((-1, x, y, 1))

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('mode'n, type=str, help='[data|train|predict]')
    parser.add_argument('-m', '--data-mode', type=str, help='[frames|sum]', default='frames')
    parser.add_argument('-o', '--output-dir', type=str, default=os.getcwd())
    parser.add_argument('-d', '--data', type=str, help='File containing data (from data mode).')
    parser.add_argument('--train-files', nargs='+', type=str, default=[],
        help='Files to train model on.')
    parser.add_argument('--test-files', nargs='+', type=str, default=[],
        help='Files to test model on.')
    parser.add_argument('--val-files', nargs='+', type=str, default=[],
        help='Files to validate model on.')
    args = parser.parse_args()
    if args.mode not in ['data', 'train', 'predict']:
        parser.print_help()
    return args

In [ ]:
# # Using data generated by drunk_detector, including augmented data (flip, noise, and blur)
# with open('data_2020-04-09_12-51-37.pickle', 'rb') as data_file:
#     data = pickle.load(data_file)

# # Use all frame data
# if False:
#     train_X = np.array([datum['thermal_frames'] for datum in data['train']])
#     val_X = np.array([datum['thermal_frames'] for datum in data['val']])
#     test_X = np.array([datum['thermal_frames'] for datum in data['test']])
    
# # Use sum data
# elif True:
#     train_X = np.array([datum['thermal_sum'] for datum in data['train']])
#     val_X = np.array([datum['thermal_sum'] for datum in data['val']])
#     test_X = np.array([datum['thermal_sum'] for datum in data['test']])

# train_X_2d = np.array([datum.flatten() for datum in train_X])
# val_X_2d = np.array([datum.flatten() for datum in val_X])
# test_X_2d = np.array([datum.flatten() for datum in test_X])

# train_y = np.array([datum['y'] for datum in data['train']])
# val_y = np.array([datum['y'] for datum in data['val']])
# test_y = np.array([datum['y'] for datum in data['test']])

In [ ]:
# # Shuffle X_train and y_train together
# from sklearn.utils import shuffle

# def shuffle_data(X, y):
#     X_s, y_s = shuffle(X, y, random_state=42)
#     return X_s, y_s

# x,y = train_X.shape[1:]
# # Get shuffled data and prepare for model.fit inputs
# X_train_s, y_train_s = shuffle_data(train_X, train_y)
# X_val_s, y_val_s = shuffle_data(val_X, val_y)
# X_test_s, y_test_s = shuffle_data(test_X, test_y)
# input_data = X_train_s.reshape((-1, x, y, 1))
# val_data = X_val_s.reshape((-1, x, y, 1))
# test_data = X_test_s.reshape((-1, x, y, 1))

In [ ]:
class Args:
    data='data_unaugmented.pickle'
    data_mode='sum'
    mode='train'
    output_dir='/home/kevin/affective_computing_project/jupyter'
    test_files=[]
    train_files=[]
    val_files=[]
    
    
args = Args()
dd = DrunkDetector(args)

if args.mode == 'data':
    dd.format_data()

elif args.mode == 'train':
    # TODO randomize train and val data here
    dd.train()

CNN hyperparameter tuning with 486 combinations
Run 0/486
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 198 samples, validate on 48 samples
Epoch 1/1000
198/198 [==============================] - 2s 9ms/sample - loss: 0.1955 - accuracy: 0.3838 - val_loss: 0.1310 - val_accuracy: 0.7500
Epoch 2/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1317 - accuracy: 0.4747 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 3/1000
198/198 [==============================] - 1s 7ms/sample - loss: 0.1306 - accuracy: 0.2525 - val_loss: 0.1302 - val_accuracy: 0.7500
Epoch 4/1000
198/198 [==============================] - 1s 7ms/sample - loss: 0.1311 - accuracy: 0.7475 - val_loss: 0.1305 - val_accuracy: 0.7500
Epoch 5/1000
198/198 [==============================] - 1s 7ms/sample - loss: 0.1311 - accuracy: 0.7475 - val_loss: 0.1304 - val_accuracy: 0.7500
Epoch 6/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1312 - accuracy: 0.7273 - val_loss: 0

198/198 [==============================] - 0s 1ms/sample - loss: 0.1246 - accuracy: 0.7121 - val_loss: 0.1293 - val_accuracy: 0.7083
Epoch 13/1000
198/198 [==============================] - 0s 1ms/sample - loss: 0.1235 - accuracy: 0.7222 - val_loss: 0.1292 - val_accuracy: 0.4375
Epoch 14/1000
198/198 [==============================] - 0s 1ms/sample - loss: 0.1226 - accuracy: 0.5051 - val_loss: 0.1303 - val_accuracy: 0.3750
Epoch 15/1000
198/198 [==============================] - 0s 1ms/sample - loss: 0.1232 - accuracy: 0.4192 - val_loss: 0.1298 - val_accuracy: 0.3333
Epoch 16/1000
198/198 [==============================] - 0s 1ms/sample - loss: 0.1220 - accuracy: 0.6465 - val_loss: 0.1293 - val_accuracy: 0.6667
Epoch 17/1000
198/198 [==============================] - 0s 1ms/sample - loss: 0.1214 - accuracy: 0.5909 - val_loss: 0.1303 - val_accuracy: 0.3333
Epoch 18/1000
198/198 [==============================] - 0s 1ms/sample - loss: 0.1198 - accuracy: 0.5354 - val_loss: 0.1297 - val_ac

198/198 [==============================] - 1s 3ms/sample - loss: 0.1090 - accuracy: 0.6010 - val_loss: 0.1456 - val_accuracy: 0.5000
Epoch 21/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1026 - accuracy: 0.7121 - val_loss: 0.1515 - val_accuracy: 0.4167
Epoch 22/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1054 - accuracy: 0.6414 - val_loss: 0.1489 - val_accuracy: 0.6667
Epoch 23/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1081 - accuracy: 0.7525 - val_loss: 0.1499 - val_accuracy: 0.3750
Epoch 24/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1003 - accuracy: 0.6414 - val_loss: 0.1449 - val_accuracy: 0.5625
Epoch 25/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1006 - accuracy: 0.7778 - val_loss: 0.1524 - val_accuracy: 0.3958
Epoch 26/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.0976 - accuracy: 0.7121 - val_loss: 0.1448 - val_ac

Epoch 76/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.1595 - val_accuracy: 0.6667
Epoch 77/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.1678 - val_accuracy: 0.6667
Epoch 78/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.1566 - val_accuracy: 0.6667
Epoch 79/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.1611 - val_accuracy: 0.6458
Epoch 80/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.1626 - val_accuracy: 0.6458
Epoch 81/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.1748 - val_accuracy: 0.6667
Epoch 82/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 198 samples, validate on 48 samples
Epoch 1/1000
198/198 [==============================] - 1s 5ms/sample - loss: 0.1312 - accuracy: 0.5152 - val_loss: 0.1392 - val_accuracy: 0.7500
Epoch 2/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1330 - accuracy: 0.5606 - val_loss: 0.1301 - val_accuracy: 0.2500
Epoch 3/1000
198/198 [==============================] - 0s 3ms/sample - loss: 0.1305 - accuracy: 0.2525 - val_loss: 0.1301 - val_accuracy: 0.2500
Epoch 4/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1308 - accuracy: 0.2525 - val_loss: 0.1302 - val_accuracy: 0.2500
Epoch 5/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1309 - accuracy: 0.2525 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 6/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1304 - accuracy: 0.2525 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 7/1000
198/198 [=======

...

198/198 [==============================] - 1s 3ms/sample - loss: 1.4574e-04 - accuracy: 1.0000 - val_loss: 0.2149 - val_accuracy: 0.6458
Epoch 32/1000
198/198 [==============================] - 1s 3ms/sample - loss: 1.2971e-04 - accuracy: 1.0000 - val_loss: 0.2230 - val_accuracy: 0.6458
Epoch 33/1000
198/198 [==============================] - 1s 3ms/sample - loss: 1.0302e-04 - accuracy: 1.0000 - val_loss: 0.2275 - val_accuracy: 0.6458
Epoch 34/1000
198/198 [==============================] - 1s 3ms/sample - loss: 9.0057e-05 - accuracy: 1.0000 - val_loss: 0.2288 - val_accuracy: 0.6458
Epoch 00034: early stopping
Run 103/486
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 198 samples, validate on 48 samples
Epoch 1/1000
198/198 [==============================] - 1s 3ms/sample - loss: 0.1313 - accuracy: 0.2576 - val_loss: 0.1301 - val_accuracy: 0.2500
Epoch 2/1000
198/198 [==============================] - 0s 1ms/sample - loss: 0.1304 - accuracy: 0.2525 - val_loss: 0.1300 - val_

198/198 [==============================] - 0s 3ms/sample - loss: 0.1305 - accuracy: 0.2525 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 34/1000
198/198 [==============================] - 0s 2ms/sample - loss: 0.1305 - accuracy: 0.2525 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 35/1000
198/198 [==============================] - 0s 2ms/sample - loss: 0.1304 - accuracy: 0.2525 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 36/1000
198/198 [==============================] - 0s 2ms/sample - loss: 0.1304 - accuracy: 0.2525 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 00036: early stopping
Run 106/486
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 198 samples, validate on 48 samples
Epoch 1/1000
198/198 [==============================] - 1s 7ms/sample - loss: 0.1314 - accuracy: 0.3030 - val_loss: 0.1301 - val_accuracy: 0.2500
Epoch 2/1000
198/198 [==============================] - 0s 2ms/sample - loss: 0.1307 - accuracy: 0.6364 - val_loss: 0.1300 - val_accuracy: 0.7500

Epoch 31/1000
198/198 [==============================] - 3s 16ms/sample - loss: 0.1304 - accuracy: 0.2525 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 00031: early stopping
Run 109/486
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 198 samples, validate on 48 samples
Epoch 1/1000
198/198 [==============================] - 2s 8ms/sample - loss: 0.1845 - accuracy: 0.5101 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 2/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1328 - accuracy: 0.5303 - val_loss: 0.1301 - val_accuracy: 0.2500
Epoch 3/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1309 - accuracy: 0.2525 - val_loss: 0.1300 - val_accuracy: 0.2500
Epoch 4/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1305 - accuracy: 0.3737 - val_loss: 0.1300 - val_accuracy: 0.7500
Epoch 5/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1305 - accuracy: 0.7475 - val_loss: 0.1300 - val_accu

198/198 [==============================] - 1s 6ms/sample - loss: 0.1216 - accuracy: 0.7121 - val_loss: 0.1325 - val_accuracy: 0.5417
Epoch 17/1000
198/198 [==============================] - 1s 5ms/sample - loss: 0.1193 - accuracy: 0.6465 - val_loss: 0.1330 - val_accuracy: 0.5833
Epoch 18/1000
198/198 [==============================] - 1s 5ms/sample - loss: 0.1177 - accuracy: 0.5960 - val_loss: 0.1345 - val_accuracy: 0.3750
Epoch 19/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1162 - accuracy: 0.5758 - val_loss: 0.1348 - val_accuracy: 0.5833
Epoch 20/1000
198/198 [==============================] - 1s 5ms/sample - loss: 0.1156 - accuracy: 0.6970 - val_loss: 0.1364 - val_accuracy: 0.5833
Epoch 21/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1143 - accuracy: 0.5859 - val_loss: 0.1386 - val_accuracy: 0.3750
Epoch 22/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1124 - accuracy: 0.5960 - val_loss: 0.1399 - val_ac

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 198 samples, validate on 48 samples
Epoch 1/1000
198/198 [==============================] - 2s 8ms/sample - loss: 0.1304 - accuracy: 0.6313 - val_loss: 0.1300 - val_accuracy: 0.5000
Epoch 2/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1308 - accuracy: 0.7121 - val_loss: 0.1301 - val_accuracy: 0.7500
Epoch 3/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1301 - accuracy: 0.5505 - val_loss: 0.1301 - val_accuracy: 0.3750
Epoch 4/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1298 - accuracy: 0.4394 - val_loss: 0.1301 - val_accuracy: 0.4167
Epoch 5/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1295 - accuracy: 0.4545 - val_loss: 0.1301 - val_accuracy: 0.3542
Epoch 6/1000
198/198 [==============================] - 1s 6ms/sample - loss: 0.1288 - accuracy: 0.5202 - val_loss: 0.1300 - val_accuracy: 0.6458
Epoch 7/1000
198/198 [=======

In [ ]:
# Hyperparameter tuning for CNN where hyperparameters are layer sizes and batch size

# import tensorflow as tf
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.models import load_model, save_model
# from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
# from tensorflow.keras.layers import Conv2D, MaxPooling2D
# from tensorflow.keras import optimizers
# import tensorflow.keras.metrics


# from sklearn.model_selection import KFold

# import matplotlib.pyplot as plt

# import itertools
# import copy

# np.random.seed(42)

# # val scores from each run will be appended
# # saved_acc = []
# saved_acc = pickle.load(open('saved_acc.pkl', 'rb'))

# # hyperparameters
# layer1_size = [2,8,32,128]
# layer2_size = [2,8,32,128]
# layer3_size = [2,8,32,128]
# layer4_size = [2,8,32,128]
# batch_size = [2,8,32,128]

# # creates a list of all combinations of hyperparameters
# param_grid = list(itertools.product(layer1_size, layer2_size, layer3_size, layer4_size, batch_size))

# offset = len(saved_acc)
# for i, params in enumerate(param_grid[offset:]):
#     i = i + offset
#     # 32, 64 works well on validation data
#     # 8. 16 also works well on validation data
#     # 8,32,32 works well on test data
#     ## Model
#     model = Sequential()

#     model.add(Conv2D(params[0], (3, 3), input_shape=(x,y,1)))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     model.add(Conv2D(params[1], (3, 3)))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     model.add(Conv2D(params[2], (3, 3)))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))


#     model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

#     model.add(Dense(params[3], activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dense(1))

#     model.add(Activation('sigmoid'))

#     ##

#     # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics
#     METRICS = [
#           tensorflow.keras.metrics.TruePositives(name='tp'),
#           tensorflow.keras.metrics.FalsePositives(name='fp'),
#           tensorflow.keras.metrics.TrueNegatives(name='tn'),
#           tensorflow.keras.metrics.FalseNegatives(name='fn'), 
#           tensorflow.keras.metrics.BinaryAccuracy(name='accuracy'),
#           tensorflow.keras.metrics.Precision(name='precision'),
#           tensorflow.keras.metrics.Recall(name='recall'),
#           tensorflow.keras.metrics.AUC(name='auc'),
#           tensorflow.keras.metrics.BinaryCrossentropy(name='bce')
#     ]

#     model.compile(loss='binary_crossentropy',
#                   optimizer=optimizers.Adam(learning_rate=args.learning_rate),
#                   metrics=['accuracy'])#, tensorflow.keras.metrics.Precision(name='precision')]) # Want to prevent false negatives (ie. predicting sober when actually drunk)

#     # Treat every sober image with same weight as 3 drunk images
#     # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#class_weights
#     class_weight = {0: (1/4)/2, 1: (3/4)/2}

#     # how many epochs (passes through the data)
#     num_epochs = 1000

#     # this will hold the performance
#     perf_time = np.zeros((num_epochs, 4))

#     # set up figure
#     fig = plt.figure()
#     ax1 = fig.add_subplot(111)

#     # how well did it work?
#     best_val = [np.inf, 0]
#     for epoch in np.arange(0,num_epochs):
#         # train an epoch at a time, visualize as we go!
#         model.fit(input_data, np.array(y_train_s), 
#                   batch_size=params[4], 
#                   epochs=1, 
#                   verbose=1, 
#                   class_weight=class_weight,
#                   validation_data=(val_data,np.array(y_val_s)),
#                   shuffle=True,
#                   use_multiprocessing=True)    
#         # check the performance on train/test/val
#         # the model.evaluate function returns the loss (position 0) and the performance (position 1)
#         val = model.evaluate(val_data, np.array(y_val_s))  # val = [val_loss, val_accuracy]  
#         new = [model.evaluate(input_data, np.array(y_train_s))[1],            
#                val[0], val[1],            
#                model.evaluate(test_data, np.array(y_test_s))[1]]    

#         # add to performance
#         perf_time[epoch,:]=new

#         # visualize
#         plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,0],'b', label='train')
#         plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,2],'r', label='validation')
#         plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,3],'g', label='test')
#         plt.legend(loc='upper left')
        

#         # is validation performance better?
#         if val[0] >= best_val[0]:
#             best_val[1] += 1
#         else:
#             best_val = [val[0], 0]
#         print (f'hyperparameters {str(params)}, ' + "epoch %d, loss %f, number %d" %(epoch, best_val[0], best_val[1]))

#         # if there hasn't been an improvement in three epochs, stop training
#         if best_val[1] > 30:
#             break
            
#     test_accuracy = new[3]
    
#     # set up figure
#     fig = plt.figure()
#     ax1 = fig.add_subplot(111)
    
#     plt.title(f'Hyperparameters: {str(params)}')
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy')
#     plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,0],'b', label='train')
#     plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,2],'r', label='validation')
#     plt.plot(np.arange(0,epoch+1),perf_time[0:epoch+1,3],'g', label='test')
#     plt.legend(loc='upper left')
    
#     plt.savefig(f'run-{i}-{str(params)}-test_acc={test_accuracy}.png')
    
#     plt.close('all') # close fig to save memory
    
    
#     # save current model and val score
#     save_model(model, f'run-{i}.hdf5')
#     saved_acc.append(copy.deepcopy(test_accuracy))
#     pickle.dump(saved_acc, open('saved_acc.pkl', 'wb'))

In [ ]:
best_index = np.argmax(saved_acc)

# best model found
model = load_model(f'run-{best_index}.hdf5')

print(f'Best hyperparameters:\t{param_grid[best_index]}')

In [ ]:
# Manually load best saved model here and test
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.models import load_model
model = load_model('models/n_clayers=1,clayer_sz=8,n_dlayers=0,dlayer_sz=512,lr=0.01,bat_size=32FINAL-test_acc=0.8714285492897034.hdf5')

y_pred = dd.model.predict_classes(cnn_data(dd.test_X))

print(f'Accuracy: {accuracy_score(dd.test_y,y_pred)}')
print(f'Confusion Matrix:\n{confusion_matrix(dd.test_y,y_pred)}')

for i,(actual,pred) in enumerate(zip(dd.test_y,y_pred)):
    print(f'Actual: {actual}, Pred: {pred}')
    
    print(dd.test_X[i])
    
    if actual==0 or actual==pred:
        display(Image.fromarray(dd.test_X[i]*1000).convert('L'))
        input('Enter for next true negative:')